# Organización del número de explotaciones agrícolas

## Limpieza de los datos

<b>Objetivo:</b> eliminar las líneas innecesarias, y organizar los encabezados de los documentos. De esta manera podremos unificar todos los datos posteriormente.

In [6]:
import os

def clean_data(input_folder, output_folder):
    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener la lista de archivos CSV en la carpeta de entrada
    csv_files = [file for file in os.listdir(input_folder) if file.endswith('.csv')]

    # Datos excepcionales
    unuseful_words = ['agrario', 'Serie;;;2020']
    headers = "Serie;Codigo_municipio;Nombre;2020\n"

    # Limpiar y guardar los files CSV en la carpeta de salida
    for file in csv_files:
        original_file = os.path.join(input_folder, file)
        clean_file = os.path.join(output_folder, file)
        with open(original_file, 'r') as f_input, open(clean_file, 'w') as f_output:
            f_output.write(headers) 
            delete_next = False
            for line in f_input:                    
                if not delete_next:
                    delete = False
                    for word in unuseful_words:
                        if word in line:
                            delete = True
                    if line.strip() == '':
                        delete = True
                    elif '-' in line:
                        line = line.replace('-', '0')
                    if not delete:
                        f_output.write(line)
                    if 'Zarzalejo' in line:
                        delete_next = True

input_folder = ".\original_data"
output_folder = ".\clean_data"

clean_data(input_folder, output_folder)


## Unión de los datos

<b>Objetivo:</b> unir todos los archivos de la carpeta en uno solo, con sus encabezados correspondientes. Guardar el <i>.csv</i> resultante en la carpeta <i>union_final</i> para poder añadir los datos al archivo final.

In [8]:
import os, csv

def join_data(folder_path, output_path):
    # Crear un diccionario para almacenar los datos unidos
    join_data = {}

    csv_files = os.listdir(folder_path)

    # Iterar sobre cada archivo en la carpeta
    final_fields = []
    for file in csv_files:
        if file.endswith('.csv'):
            with open(os.path.join(folder_path, file), 'r') as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=';')
                next(csv_reader)
                type = file.split('_')[0] + '_explotaciones_agricolas'
                if 'huertos' in file:
                    final_fields.append(f'{type}_huertos_consumo_propio')
                elif 'tierra' in file:
                    final_fields.append(f'{type}_tierra_pastos')
                else:
                    final_fields.append(f'{type}_total')
                                
                # Iterar sobre cada fila en el archivo CSV
                for row in csv_reader:
                    serie = row[0] if row else ""
                    codigo_municipio = row[1] if len(row) > 1 else ""
                    nombre = row[2] if len(row) > 2 else ""

                    # Verificar si la serie, código de municipio y nombre ya están en el diccionario
                    key = (serie, codigo_municipio, nombre)
                    if key in join_data:
                        join_data[key].append(row[3])
                    else:
                        join_data[key] = [serie, codigo_municipio, nombre, row[3]]

    # Escribir los datos unidos en un nuevo archivo CSV
    with open(output_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=';')
        # Escribir el header
        csv_writer.writerow(['Serie', 'Codigo_municipio', 'Nombre'] + final_fields)
        # Escribir los datos unidos
        for key, values in join_data.items():
            csv_writer.writerow(values)

folder_path = './clean_data'
output_path = '../union_final/explotaciones_agrarias.csv'

join_data(folder_path, output_path)

